In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
docs = TavilySearchResults(k=3).invoke({'query': 'NVDA stocks soaring? Investment'})

In [5]:
for doc in docs:
    print(doc)

{'url': 'https://finance.yahoo.com/news/63-2024-where-nvidias-soaring-160700899.html', 'content': "S&P 500\nDow 30\nNasdaq\nRussell 2000\nCrude Oil\nGold\nSilver\nEUR/USD\n10-Yr Bond\nGBP/USD\nUSD/JPY\nBitcoin USD\nCMC Crypto 200\nFTSE 100\nNikkei 225\nUp 63% in 2024, Where Will Nvidia's Soaring Stock Be in 5 Years?\n Before you buy stock in Nvidia, consider this:\nThe Motley Fool Stock Advisor analyst team just identified what they believe are the\xa010 best stocks for investors to buy now… and Nvidia wasn’t one of them. Because of the chipmaker's outsized role in the burgeoning AI industry, its operational results can give valuable clues about the state of the sector as a whole. With shares up a whopping 63% year to date, Nvidia (NASDAQ: NVDA) is still riding high on the artificial intelligence (AI) wave that boosted its shares by almost 240% in the last 12 months alone.\n See the 10 stocks\n*Stock Advisor returns as of February 26, 2024\nWill Ebiefung has no position in any of the s